### Tutorial - Generate a binary population by hand

This notebook follows the 
[tutorial](https://cosmic-popsynth.github.io/docs/stable/runpop/index.html) 
provided by COSMIC author Katie Breivik.

The process to generate a synthetic binary population, is similar to the process 
to evolve a single/multiple binaries by hand: first generate an initial 
population, then evolve it with the `Evolve` class.

An initialized binary population consists of a collection of binary systems with 
assigned primary and secondary masses, orbital periods, eccentricities, 
metallicities, and star formation histories. These parameters are randomly 
sampled from observationally motivated distribution functions.

In COSMIC, the initial sample is done through an initial binary sampler which 
works with the `InitialBinaryTable` class. There are two samplers available:

1. *independent* : initialize binaries with independent parameter distributions 
for the primary mass, mass ratio, eccentricity, separation, and binary fraction

2. *multidim* : initialize binaries with multidimensional parameter 
distributions according to 
[Moe & Di Stefano 2017](https://ui.adsabs.harvard.edu/abs/2017ApJS..230...15M/abstract)

We consider both cases below.

### independent

First import the `InitialBinaryTable` class and the independent sampler

In [1]:
from cosmic.sample.initialbinarytable import InitialBinaryTable
from cosmic.sample.sampler import independent

The `independent` sampler contains multiple models for each binary parameter. 
You can access the available models using the `independent` sampler help call:

In [2]:
help(independent.get_independent_sampler)

Help on function get_independent_sampler in module cosmic.sample.sampler.independent:

get_independent_sampler(final_kstar1, final_kstar2, primary_model, ecc_model, porb_model, SF_start, SF_duration, binfrac_model, met, size, **kwargs)
    Generates an initial binary sample according to user specified models
    
    Parameters
    ----------
    final_kstar1 : `int or list`
        Int or list of final kstar1
    
    final_kstar2 : `int or list`
        Int or list of final kstar2
    
    primary_model : `str`
        Model to sample primary mass; choices include: kroupa93, kroupa01, salpeter55, custom
        if 'custom' is selected, must also pass arguemts:
        alphas : `array`
            list of power law indicies
        mcuts : `array`
            breaks in the power laws.
        e.g. alphas=[-1.3,-2.3,-2.3],mcuts=[0.08,0.5,1.0,150.] reproduces standard Kroupa2001 IMF
    
    ecc_model : `str`
        Model to sample eccentricity; choices include: thermal, uniform, sana1

The `final_kstar1` and `final_kstar2` parameters are lists that contain the 
kstar types that you would like the final population to contain.

The final kstar is the final state of the binary system we are interested in and 
is based on the BSE kstar naming convention, see 
[Evolutionary State of the Star](https://cosmic-popsynth.github.io/docs/stable/output_info/index.html#kstar-table) 
for more information.

Thus, if you want to generate a population containing double white dwarfs with 
CO and ONe WD primaries and He-WD secondaries, the final kstar inputs would be:

In [3]:
final_kstar1 = [11, 12] 
final_kstar2 = [10]

Similar to the help for the sampler, the different models that can be used for 
each parameter to be sampled can be accessed by the help function for the 
argument. The syntax for each parameter sample is always: sample_'parameter'. 
See the example for the star formation history (SFH) below:

In [4]:
help(independent.Sample.sample_SFH)

Help on function sample_SFH in module cosmic.sample.sampler.independent:

sample_SFH(self, SF_start=13700.0, SF_duration=0.0, met=0.02, size=None)
    Sample an evolution time for each binary based on a user-specified
    time at the start of star formation and the duration of star formation.
    The default is a burst of star formation 13,700 Myr in the past.
    
    Parameters
    ----------
    SF_start : float
        Time in the past when star formation initiates in Myr
    SF_duration : float
        Duration of constant star formation beginning from SF_Start in Myr
    met : float
        metallicity of the population [Z_sun = 0.02]
        Default: 0.02
    size : int, optional
        number of evolution times to sample
        NOTE: this is set in cosmic-pop call as Nstep
    
    Returns
    -------
    tphys : array
        array of evolution times of size=size
    metallicity : array
        array of metallicities



Using the final kstar inputs above, the initial binary population is sampled as:

In [5]:
InitialBinaries, mass_singles, mass_binaries, n_singles, n_binaries = InitialBinaryTable.sampler(
    'independent', 
    final_kstar1, 
    final_kstar2, 
    binfrac_model=0.5, 
    primary_model='kroupa01', 
    ecc_model='sana12', 
    porb_model='sana12', 
    qmin=-1, m2_min=0.08, 
    SF_start=13700.0, 
    SF_duration=0.0, 
    met=0.02, 
    size=10000
)

print(InitialBinaries)

       kstar_1  kstar_2    mass_1    mass_2         porb       ecc  \
0          1.0      1.0  6.937819  5.016688  6194.972108  0.571817   
1          1.0      1.0  1.054708  0.876962     1.471888  0.085622   
2          1.0      1.0  1.361050  1.322399     2.771313  0.287468   
3          1.0      1.0  6.185087  5.742121   175.885756  0.895057   
4          1.0      1.0  7.028671  3.484833    18.127806  0.009407   
...        ...      ...       ...       ...          ...       ...   
10397      1.0      0.0  0.868077  0.681154   171.311951  0.376490   
10398      1.0      1.0  8.810424  3.836585     3.802050  0.040360   
10399      1.0      1.0  1.061403  0.948733     5.975174  0.247322   
10400      1.0      0.0  1.493910  0.689184  5177.858463  0.001071   
10401      1.0      0.0  0.925118  0.516114   757.152708  0.069111   

       metallicity   tphysf   mass0_1   mass0_2  ...  tacc_1  tacc_2  epoch_1  \
0             0.02  13700.0  6.937819  5.016688  ...     0.0     0.0      0.0 

NOTE: the length of the initial binary data set, `InitialBinaries`, does not 
always match the size parameter provided to `InitialBinaryTable.sampler`. This 
is because the sampler accounts for a binary fraction specified by the user with 
the `binfrac_model` parameter, which is either a fraction between 0 and 1 or 
mass dependent following the prescription in 
[van Haaften+2013](http://adsabs.harvard.edu/abs/2012A%26A...537A.104V).

If you want to have separate binary fractions and mass pairings for low and high 
mass stars, you can by supplying the `msort` kwarg to the sampler. This sets the 
mass above which an alternative mass pairing (specified by kwargs `qmin_msort` 
and `m2_min_msort`) and binary fraction model (specified by kwarg 
`binfrac_model_msort`) are used. This is handy if you want, for example, a 
higher binary fraction and more equal mass pairings for high mass stars.

Since we are interested in binaries, we only retain the binary systems that are 
likely to produce the user specified final kstar types. However, we also keep 
track of the total mass of the single and binary stars as well as the number of 
binary and single stars so that we can scale our results to larger populations. 
If you don’t want to filter the binaries, you can supply final kstars as

In [6]:
import numpy as np

final_kstars = np.linspace(0, 14, 15)

InitialBinaries, mass_singles, mass_binaries, n_singles, n_binaries = InitialBinaryTable.sampler(
    'independent', 
    final_kstars, 
    final_kstars, 
    binfrac_model=0.5, 
    primary_model='kroupa01', 
    ecc_model='sana12', 
    porb_model='sana12', 
    qmin=-1, 
    m2_min=0.08, 
    SF_start=13700.0, 
    SF_duration=0.0, 
    met=0.02, 
    size=10000
)

### multidim

COSMIC implements multidimensionally distributed initial binaries according to 
[Moe & Di Stefano 2017](https://ui.adsabs.harvard.edu/abs/2017ApJS..230...15M/abstract). 
The python code used in COSMIC to create this sample was written by Mads Sorenson, and is based on the IDL codes written to accompany Moe & Di Stefano 2017.